# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [95]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np
from __future__ import division


In [2]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [30]:
# frequency table for prestige and whether or not someone was admitted
pd.crosstab(df['prestige'], df['admit'], rownames=['prestige'], colnames=['admitted'])

admitted,0,1
prestige,,
1.0,28,33
2.0,95,53
3.0,93,28
4.0,55,12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [75]:
dummy_ranks = pd.get_dummies(df['prestige'], prefix='prestige')
dummy_ranks.head()

,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: 

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [76]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
print handCalc.head()

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             0             1             0
1      1  660.0  3.67             0             0             1             0
2      1  800.0  4.00             1             0             0             0
3      1  640.0  3.19             0             0             0             1
4      0  520.0  2.93             0             0             0             1


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [102]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
ctab1 = pd.crosstab(handCalc['admit'], handCalc['prestige_1.0'], rownames=['admitted'], colnames=['prestige 1'])
ctab1

prestige 1,0,1
admitted,,
0,243,28
1,93,33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [103]:
p1_odds = ctab1.loc[1,1]/ctab1.loc[0,1]
print 'the odds of getting into grad school from a rank 1 alma mater is', p1_odds

the odds of getting into grad school from a rank 1 alma mater is 1.17857142857


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [106]:
np1_odds = ctab1.loc[1,0]/ctab1.loc[0,0]

print 'the odds of geting into grad school from a non-rank 1 alma mater is', np1_odds

the odds of geting into grad school from a non-rank 1 alma mater is 0.382716049383


#### 3.3 Calculate the odds ratio

In [107]:
odds_ratio = p1_odds / np1_odds
print 'The odds ratio is', odds_ratio

The odds ratio is 3.07949308756


#### 3.4 Write this finding in a sentenance: 

Answer: People from a rank 1 prestige undergrad school have roughly 3.08 times the odds of getting into grad school than those who are not from a rank 1 prestige undergrad school

#### 3.5 Print the cross tab for prestige_4

In [108]:
df_p4 = handCalc[['admit', 'prestige_4.0']]
ctab4 = pd.crosstab(df_p4['admit'], df_p4['prestige_4.0'], rownames=['admitted'], colnames=['prestige 4'])
ctab4

prestige 4,0,1
admitted,,
0,216,55
1,114,12


#### 3.6 Calculate the OR 

In [111]:
p4_trueodds = ctab4.loc[1,1]/ctab4.loc[0,1]
print p4_trueodds

p4_falseodds = ctab4.loc[1,0]/ctab4.loc[0,0]
print p4_falseodds

0.218181818182
0.527777777778


In [113]:
p4_odds_ratio = p4_trueodds/p4_falseodds
p4_odds_ratio

0.41339712918660282

#### 3.7 Write this finding in a sentence

Answer: People from rank 4 prestige undergrad schools have 0.41 times the odds of getting into grad school than those who did not go to a rank 4 prestige undergrad school.

## Part 4. Analysis

In [114]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             1             0
1      1  660.0  3.67             0             1             0
2      1  800.0  4.00             0             0             0
3      1  640.0  3.19             0             0             1
4      0  520.0  2.93             0             0             1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [117]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [118]:
train_columns = ['gre', 'gpa', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0', 'intercept']
#I will name my variables how i want! 

#### 4.2 Fit the model

In [121]:
X = data[train_columns]
y = data['admit']

logit = sm.Logit(y, X)
summ = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [122]:
summ.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Mon, 02 Apr 2018   Pseudo R-squ.:                 0.08166
Time:                        01:41:38   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa              0.7793      0.333      2.344      0.019       0.128       1.431
prestige_2.0    -0.6801      0.317     -2.146      0.032      -1.301      -0.059
prestige_3.0    -1.3387      0.345     -3.882      0.000      -2.015      -0.663
prestige_4.0    -1.5534      0.417     -3.721      0.000      -2.372      -0.735
intercept       -3.8769      1.142     -3.393      0.001      -6.116      -1.638
================================================================================
"""

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [124]:
np.exp(summ.params)

gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64

In [126]:
params = summ.params
conf_int = summ.conf_int()
conf_int['OR'] = params
conf_int.columns = ['2.5%', '97.5%', 'OR']

conf_int = np.exp(conf_int)
conf_int

,2.5%,97.5%,OR
gre,1.000074,1.004372,1.002221
gpa,1.136120,4.183113,2.180027
prestige_2.0,0.272168,0.942767,0.506548
prestige_3.0,0.133377,0.515419,0.262192
prestige_4.0,0.093329,0.479411,0.211525
intercept,0.002207,0.194440,0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer: The log of the odds of being admitted to grad school is decreased by 0.51 if from a prestige_2 school instead of a prestige_1 school

#### 4.6 Interpret the OR of GPA

Answer: The log of the odds of being admitted increases by 2.18 for each point of increase in GPA

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [133]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n // arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [129]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


In [135]:
# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))
combos.head()

,0,1,2,3
0,220.0,2.260000,1.0,1.0
1,220.0,2.260000,2.0,1.0
2,220.0,2.260000,3.0,1.0
3,220.0,2.260000,4.0,1.0
4,220.0,2.453333,1.0,1.0


#### 5.1 Recreate the dummy variables

In [138]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
prest_dummies = pd.get_dummies(combos['prestige'], prefix='prestige')

# keep only what we need for making predictions
keep_column = ['gre', 'gpa','intercept']
combos = combos[keep_column].join(prest_dummies.loc[:,'prestige_2.0':])
combos.head()

,gre,gpa,intercept,prestige_2.0,prestige_3.0,prestige_4.0
0,220.0,2.260000,1.0,0,0,0
1,220.0,2.260000,1.0,1,0,0
2,220.0,2.260000,1.0,0,1,0
3,220.0,2.260000,1.0,0,0,1
4,220.0,2.453333,1.0,0,0,0


#### 5.2 Make predictions on the enumerated dataset

In [140]:
combos['prob_predict'] = summ.predict(combos[train_columns])
combos.head()

,gre,gpa,intercept,prestige_2.0,prestige_3.0,prestige_4.0,prob_predict
0,220.0,2.260000,1.0,0,0,0,0.164173
1,220.0,2.260000,1.0,1,0,0,0.090492
2,220.0,2.260000,1.0,0,1,0,0.048977
3,220.0,2.260000,1.0,0,0,1,0.039890
4,220.0,2.453333,1.0,0,0,0,0.185907


#### 5.3 Interpret findings for the last 4 observations

In [141]:
combos.tail(4)

,gre,gpa,intercept,prestige_2.0,prestige_3.0,prestige_4.0,prob_predict
396,800.0,4.0,1.0,0,0,0,0.734040
397,800.0,4.0,1.0,1,0,0,0.582995
398,800.0,4.0,1.0,0,1,0,0.419833
399,800.0,4.0,1.0,0,0,1,0.368608


Answer: These last 4 observations show 4 different people who all have perfect gpas and gre scores. The difference in the values of their "probability predicted" column is due to the difference in prestige. As one can see, the higher the prestige (meaning closer to rank 1), the higher the probability of being admitted jumps. Conversely, the lower the prestige (meaning closer to rank 4), the lower the probability of being admitted drops. According to my model, rank plays a fairly large role in the odds of admission.

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.